In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from  sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns

In [19]:
# Depression scores from questionnaire of 2005-2018
files = ["DPQ_D.XPT", "DPQ_E.XPT", "DPQ_F.XPT", "DPQ_G.XPT", "DPQ_H.XPT", "DPQ_I.XPT", "DPQ_J.XPT"]

columns = ["DPQ0" + str(i) for i in range(10, 100, 10)]  # DPQ010 to DPQ090
columns.insert(0,'SEQN')  # add SEQN column
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/questionnaire/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)

# combine all dataframes
combined_df = pd.concat(dfs)
combined_df = combined_df[(combined_df != 7) & (combined_df != 9)].dropna()
combined_df = np.round(combined_df, 0).astype('int64')
combined_df['score'] = combined_df.sum(axis=1) - combined_df['SEQN']
combined_df = combined_df[['SEQN', 'score']]

In [20]:
# complete blood count
files = ["CBC_D.XPT", "CBC_E.XPT", "CBC_F.XPT", "CBC_G.XPT", "CBC_H.XPT", "CBC_I.XPT", "CBC_J.XPT"]
columns = ["SEQN","LBXWBCSI","LBXLYPCT","LBXMOPCT", "LBXNEPCT", "LBXEOPCT", "LBXBAPCT", "LBXRBCSI", "LBXHGB", "LBXMCVSI", "LBXRDW", "LBXPLTSI","LBXMPSI"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/CBC/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
CBC_df = pd.concat(dfs)
CBC_df = CBC_df.dropna()
# convert SEQN to int
CBC_df['SEQN'] = CBC_df['SEQN'].astype('int64')
# merge with combined_df
combined_df = pd.merge(combined_df, CBC_df, on='SEQN')

In [21]:
# demographics
files = ["DEMO_D.XPT", "DEMO_E.XPT", "DEMO_F.XPT", "DEMO_G.XPT", "DEMO_H.XPT", "DEMO_I.XPT", "DEMO_J.XPT"]
columns = ["SEQN","RIAGENDR", "RIDAGEYR", "DMDEDUC2"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/demo/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
demo_df = pd.concat(dfs)
demo_df = demo_df.dropna()
demo_df = demo_df[(demo_df != 7) & (demo_df != 9)].dropna()
demo_df = np.round(demo_df, 0).astype('int64')
demo_df['male'] = (demo_df['RIAGENDR'] == 1)*1
demo_df['female'] = (demo_df['RIAGENDR'] == 2)*1
demo_df = demo_df.drop(columns=['RIAGENDR'])
demo_df.rename(columns={'RIDAGEYR': 'age', 'DMDEDUC2': 'education'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, demo_df, on='SEQN')

In [22]:
# sleep 2005-2014
files = ["SLQ_D.XPT", "SLQ_E.XPT", "SLQ_F.XPT", "SLQ_G.XPT", "SLQ_H.XPT"]
columns = ['SEQN', 'SLD010H']
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/sleep/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
# 2015-2016
df = pd.read_sas('data/sleep/' + 'SLQ_I.XPT',format='xport',encoding='utf-8')  # read the file
df = df[["SEQN", 'SLD012']] # extract the required columns
df.rename(columns={'SLD012': 'SLD010H'}, inplace=True)
dfs.append(df)

df = pd.read_sas('data/sleep/' + 'SLQ_J.XPT',format='xport',encoding='utf-8')  # read the file
df = df[["SEQN", 'SLD012']] # extract the required columns
df.rename(columns={'SLD012': 'SLD010H'}, inplace=True)
dfs.append(df)

sleep_df = pd.concat(dfs)
sleep_df = sleep_df[(sleep_df != 77) & (sleep_df != 99)].dropna()
sleep_df = np.round(sleep_df, 0).astype('int64')
sleep_df.rename(columns={'SLD010H': 'sleep'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, sleep_df, on='SEQN')

In [23]:
# alcohol
files = ["ALQ_D.XPT", "ALQ_E.XPT", "ALQ_F.XPT", "ALQ_G.XPT", "ALQ_H.XPT", "ALQ_I.XPT", "ALQ_J.XPT"]
columns = ['SEQN', 'ALQ130']
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/alcohol/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)

alcohol_df = pd.concat(dfs)
alcohol_df = alcohol_df[(alcohol_df != 777) & (alcohol_df != 999)].dropna()
alcohol_df = np.round(alcohol_df, 0).astype('int64')
alcohol_df.rename(columns={'ALQ130': 'alcohol'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, alcohol_df, on='SEQN')

In [24]:
# diet
files = ["DBQ_D.XPT", "DBQ_E.XPT", "DBQ_F.XPT", "DBQ_G.XPT", "DBQ_H.XPT", "DBQ_I.XPT", "DBQ_J.XPT"]
columns = ["SEQN", "DBQ700"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/diet/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
    
diet_df = pd.concat(dfs)
diet_df = diet_df[(diet_df != 7) & (diet_df != 9)].dropna()
diet_df = np.round(diet_df, 0).astype('int64')
# how healthy is diet: 5 is best, 1 is worst
diet_df['DBQ700'] = 6 - diet_df['DBQ700']
diet_df.rename(columns={'DBQ700': 'diet'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, diet_df, on='SEQN')

In [25]:
# physical activity
files = ["PAQ_E.XPT", "PAQ_F.XPT", "PAQ_G.XPT", "PAQ_H.XPT", "PAQ_I.XPT", "PAQ_J.XPT"]
columns = ["SEQN","PAQ650", "PAQ665"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/physical_activity/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
# 2005-2006
df = pd.read_sas('data/physical_activity/' + 'PAQ_D.XPT',format='xport',encoding='utf-8')  # read the file
df = df[["SEQN","PAD200", "PAD320"]] # extract the required columns
# change the value 3 to 2
df['PAD200'] = df['PAD200'].replace(3, 2)
df['PAD320'] = df['PAD320'].replace(3, 2)
df.rename(columns={'PAD200': 'PAQ650', 'PAD320': 'PAQ665'}, inplace=True)
dfs.append(df)

    
physical_activity_df = pd.concat(dfs)
physical_activity_df = physical_activity_df[(physical_activity_df != 7) & (physical_activity_df != 9)].dropna()
physical_activity_df = np.round(physical_activity_df, 0).astype('int64')
physical_activity_df.replace(2, 0, inplace=True)
physical_activity_df.rename(columns={'PAQ650': 'vigorous activity', 'PAQ665': 'moderate activity'}, inplace=True)
# sort by SEQN in ascending order
physical_activity_df.sort_values(by=['SEQN'], inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, physical_activity_df, on='SEQN')



In [26]:
combined_df

,SEQN,score,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBXRBCSI,LBXHGB,...,LBXMPSI,age,education,male,female,sleep,alcohol,diet,vigorous activity,moderate activity
0,31132,0,7.5,29.4,9.1,58.9,2.2,0.4,4.72,14.5,...,8.6,70,5,1,0,7,1,4,0,1
1,31134,0,6.6,29.1,12.0,55.4,3.3,0.2,5.45,15.9,...,7.8,73,3,1,0,7,2,3,0,1
2,31144,0,9.6,22.1,7.3,65.7,3.7,1.3,5.09,17.0,...,9.1,21,3,1,0,8,2,5,1,1
3,31150,1,6.5,18.8,9.6,68.7,2.4,0.4,4.48,14.4,...,8.8,79,3,1,0,8,3,2,1,1
4,31154,0,6.9,36.0,7.7,52.0,3.3,1.0,5.41,16.5,...,8.4,62,3,1,0,5,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21917,102943,1,9.5,44.3,4.6,47.9,2.1,1.2,4.69,13.9,...,9.3,48,4,0,1,6,2,1,0,0
21918,102944,0,7.6,35.6,8.1,50.3,5.6,0.5,5.31,16.9,...,6.7,55,3,1,0,6,3,4,0,0
21919,102947,1,3.6,38.0,18.6,42.5,0.3,0.7,4.93,10.7,...,9.3,75,5,1,0,7,1,4,0,1
21920,102953,3,8.3,28.3,6.8,60.8,3.6,0.6,5.01,15.5,...,8.7,42,3,1,0,6,12,1,0,0


In [37]:
# smoking
files = ["SMQ_D.XPT","SMQ_E.XPT", "SMQ_F.XPT", "SMQ_G.XPT", "SMQ_H.XPT", "SMQ_I.XPT", "SMQ_J.XPT"]
columns = ["SEQN", "SMQ040"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/smoking/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)

smoking_df = pd.concat(dfs)
smoking_df = smoking_df[(smoking_df != 7) & (smoking_df != 9)].dropna()
smoking_df = np.round(smoking_df, 0).astype('int64')
smoking_df.rename(columns={'SMQ040': 'smoking'}, inplace=True)
# change  3 to 0, 2 to 1, and 1 to 2
smoking_df['smoking'] = smoking_df['smoking'].replace(3, 0)
smoking_df['smoking'] = smoking_df['smoking'].replace(2, 3)
smoking_df['smoking'] = smoking_df['smoking'].replace(1, 2) #temp
smoking_df['smoking'] = smoking_df['smoking'].replace(3, 1)
# merge with combined_df
combined_df = pd.merge(combined_df, smoking_df, on='SEQN')

In [40]:
# Blood pressure
files = ["BPX_D.XPT", "BPX_E.XPT", "BPX_F.XPT", "BPX_G.XPT", "BPX_H.XPT", "BPX_I.XPT", "BPX_J.XPT"]
columns = ["SEQN", "BPXSY1", "BPXDI1"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/BP/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
    
blood_pressure_df = pd.concat(dfs)
blood_pressure_df = blood_pressure_df.dropna()
blood_pressure_df = np.round(blood_pressure_df, 0).astype('int64')
blood_pressure_df.rename(columns={'BPXSY1': 'SBP', 'BPXDI1': 'DBP'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, blood_pressure_df, on='SEQN')

In [46]:
# weight [kG]
files = ["BMX_D.XPT", "BMX_E.XPT", "BMX_F.XPT", "BMX_G.XPT", "BMX_H.XPT", "BMX_I.XPT", "BMX_J.XPT"]
columns = ["SEQN", "BMXWT"]
dfs = []  # list to store dataframes

for file in  files:
    df = pd.read_sas('data/weight/' + file,format='xport',encoding='utf-8')  # read the file
    df = df[columns] # extract the required columns
    dfs.append(df)
    
weight_df = pd.concat(dfs)
weight_df = weight_df.dropna()

weight_df.rename(columns={'BMXWT': 'weight'}, inplace=True)
# merge with combined_df
combined_df = pd.merge(combined_df, weight_df, on='SEQN')


In [47]:
combined_df

,SEQN,score,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBXRBCSI,LBXHGB,...,female,sleep,alcohol,diet,vigorous activity,moderate activity,smoking,SBP,DBP,weight
0,31150,1,6.5,18.8,9.6,68.7,2.4,4.000000e-01,4.48,14.4,...,0,8,3,2,1,1,0,144,74,85.0
1,31154,0,6.9,36.0,7.7,52.0,3.3,1.000000e+00,5.41,16.5,...,0,5,3,3,0,0,2,154,80,88.2
2,31158,2,8.0,25.6,5.9,62.2,5.1,1.200000e+00,4.73,15.9,...,0,9,2,5,0,1,2,154,74,69.2
3,31167,3,9.5,27.3,7.6,62.3,2.3,4.000000e-01,5.30,16.3,...,0,4,2,3,0,1,2,144,80,129.9
4,31192,2,10.6,13.0,5.1,81.4,0.6,5.397605e-79,4.65,13.1,...,1,6,1,1,1,1,0,128,76,91.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10027,102920,7,6.1,31.9,8.1,55.0,4.2,8.000000e-01,4.79,14.6,...,1,7,3,2,0,0,0,166,94,73.0
10028,102924,6,11.3,32.6,8.0,57.8,0.8,1.000000e+00,4.87,13.5,...,1,6,1,4,0,0,0,124,68,129.2
10029,102947,1,3.6,38.0,18.6,42.5,0.3,7.000000e-01,4.93,10.7,...,0,7,1,4,0,1,0,160,82,73.2
10030,102953,3,8.3,28.3,6.8,60.8,3.6,6.000000e-01,5.01,15.5,...,0,6,12,1,0,0,0,124,76,97.4
